# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазин обратился к нам, чтобы создать инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

**Цель:** обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Руководство магазина требует от нас построить модель со значением метрики качества *F1* не меньше 0.75. 

**Задачи:**

1. Загрузить и подготовить данные.
2. Обучить разные модели. 
3. Сделать выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

<h1>Содержание<span class="tocSkip"></span></h1>
<a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Модель-логистической-регрессии" data-toc-modified-id="Модель-логистической-регрессии-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Модель логистической регрессии</a></span></li><li><span><a href="#Модель-решающего-дерева" data-toc-modified-id="Модель-решающего-дерева-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Модель решающего дерева</a></span></li><li><span><a href="#Модель-случайного-леса" data-toc-modified-id="Модель-случайного-леса-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Модель случайного леса</a></span></li><li><span><a href="#Модель-градиентного-бустинга" data-toc-modified-id="Модель-градиентного-бустинга-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Модель градиентного бустинга</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li>

## Подготовка

In [47]:
# Импортируем необходимые библиотеки
import numpy as np
import pandas as pd
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from sklearn.pipeline import Pipeline
import re
import nltk 
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

In [48]:
# Загрузим данные и посмотрим на первые 10 строк
df = pd.read_csv('/datasets/toxic_comments.csv')
df.head(10)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


In [49]:
print('Количество строк в таблице составляет', df.shape[0])

Количество строк в таблице составляет 159571


In [50]:
# Проверим данные на наличие пропусков
df.isna().sum()

text     0
toxic    0
dtype: int64

Пропусков в данных нет. Проверим далее на наличие дубликатов.

In [51]:
print('Количество дубликатов:', df.duplicated().sum())

Количество дубликатов: 0


Займемся предобработкой текста. Начнем с приведения символов к нижнему регистру.

In [52]:
df['text'] = df['text'].str.lower()

Поскольку мы имеем дело с очень большим датасетом (159571 строк), то лемматизация будет проходить очень долго. Поэтому возьмем 50000 случайных строк.

In [53]:
df_new = df.sample(n=50000, random_state=12345).reset_index(drop=True)

Создадим корпус постов. Преобразуем столбец text в тестовый формат.

In [54]:
corpus = df_new['text'].values.astype('U')

Теперь очистим текст от ненужных символов, чтоб в дальнейшем они не мешали процессу обучения модели.

In [55]:
# Составим функцию для очистки текста
def clear_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.split()
    text = ' '.join(text)
    
    return text

In [56]:
# Применим функцию к нашему тексту
for i in range(len(corpus)):
    corpus[i] = clear_text(corpus[i])

In [57]:
# Составим функцию для токенизации и лемматизации текста
def lemmatizered(text):
    wnl = WordNetLemmatizer()
    corpus_new = []
    for sentence in text:
        word_list = nltk.word_tokenize(sentence)
        corpus_new.append(' '.join([wnl.lemmatize(w) for w in word_list]))
    return corpus_new

In [58]:
# Применим функцию к нашему корпусу
lem = lemmatizered(corpus)

In [59]:
# Добавим в исходную таблицу лемматизированный текст
df_corpus = pd.DataFrame(lem)
df_new['lemm_text'] = df_corpus[0]
df_new.head(10)

,text,toxic,lemm_text
0,ahh shut the fuck up you douchebag sand nigger...,1,ahh shut the fuck up you douchebag sand nigger...
1,"""\n\nreply: there is no such thing as texas co...",0,reply there is no such thing a texas commerce ...
2,"reply\nhey, you could at least mention jasenov...",0,reply hey you could at least mention jasenovac...
3,"thats fine, there is no deadline ) chi?",0,thats fine there is no deadline chi
4,"""\n\ndyk nomination of mustarabim\n hello! you...",0,dyk nomination of mustarabim hello your submis...
5,"""\n\nsockpuppetry case\n \nyou have been accus...",0,sockpuppetry case you have been accused of soc...
6,"judging by what i've just read in an article, ...",0,judging by what i ve just read in an article t...
7,todd and copper\nin the first film they were l...,0,todd and copper in the first film they were li...
8,"""\n\n \nyou have been blocked from editing for...",0,you have been blocked from editing for a perio...
9,| decline=can't find evidence of block either ...,0,decline can t find evidence of block either a ...


**Вывод**
На первом этапе был загружен и обработан текст. В процессе предобработки текста было выбрано случайным образом 20000 строк для облегчения процесса лемматизации и дальнейшего обучения модели. Также была проведена очистка текста от ненужных символов, его токенизация и лемматизация. Для обучения моделей применим величины TF-IDF как признаки, позволяющие определить тональность текста.

## Обучение

In [60]:
# Выделим фич и таргет
features = df_new['lemm_text']
target = df_new['toxic']

In [61]:
# Разобьем исходные данные на обучающую и тестовую выборки
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345, stratify = df_new['toxic'])

In [62]:
# Выделим стоп-слова для применения TF-IDF 
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
# Преобразуем текст в векторы с помощью TfidfVectorizer
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf_model = count_tf_idf.fit(features_train)

train_tfidf = count_tf_idf.transform(features_train)
test_tfidf = count_tf_idf.transform(features_test)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Для обучения будем использовать модели логистической регрессии, решающего дерева, случайного леса и градиентного бустинга.

### Модель логистической регрессии

С помощью GridSearchCV найдем оптимальные гиперпараметры модели.

In [63]:
%%time

lr = LogisticRegression(penalty='l2', class_weight='balanced',  solver='liblinear', random_state=12345)
parameters = {'C': [0.001,0.01,0.1,1,10,100,1000]}
lr_grid = GridSearchCV(lr, parameters, cv=3, scoring='f1')
lr_grid.fit(train_tfidf, target_train)
# Найдем оптимальные гиперпараметры
lr_grid.best_params_

CPU times: user 42.8 s, sys: 50 s, total: 1min 32s
Wall time: 1min 32s


{'C': 10}

Вычислим значение f1 на тестовой выборке при C = 10.

In [64]:
%%time

lr = LogisticRegression(C=10, penalty='l2', class_weight='balanced',  solver='liblinear', random_state=12345)
lr.fit(train_tfidf, target_train)
predictions = lr.predict(test_tfidf)
print('Значение f1 при использовании модели логистической регрессии:', round(f1_score(target_test, predictions), 2))

Значение f1 при использовании модели логистической регрессии: 0.75
CPU times: user 3.21 s, sys: 3.08 s, total: 6.29 s
Wall time: 6.3 s


### Модель решающего дерева

Аналогично предыдущему пункту 2.1 найдем оптимальные гиперпараметры для модели решающего дерева.

In [65]:
%%time

dtc = DecisionTreeClassifier(class_weight='balanced', random_state=12345)
parameters = {'max_depth': range(1, 13, 1)}
dtc_grid = GridSearchCV(dtc, parameters, cv=3, scoring='f1')
dtc_grid.fit(train_tfidf, target_train)
dtc_grid.best_params_

CPU times: user 49.2 s, sys: 0 ns, total: 49.2 s
Wall time: 49.7 s


{'max_depth': 12}

Вычислим значение f1 на тестовой выборке при max_depth = 12.

In [66]:
%%time

dtc = DecisionTreeClassifier(max_depth=12, class_weight='balanced', random_state=12345)
dtc.fit(train_tfidf, target_train)
predictions = dtc.predict(test_tfidf)
print('Значение f1 при использовании модели решающего дерева:', round(f1_score(target_test, predictions), 2))

Значение f1 при использовании модели решающего дерева: 0.59
CPU times: user 1.8 s, sys: 0 ns, total: 1.8 s
Wall time: 1.81 s


### Модель случайного леса

Найдем оптимальные гиперпараметры для модели случайного леса.

In [67]:
%%time

rfc = RandomForestClassifier(class_weight='balanced', random_state=12345)
parameters = {'n_estimators': range(10, 50, 5),
              'max_depth': range(1, 10, 2),
              'min_samples_leaf': range(3, 5),
              'min_samples_split': range(2, 6, 2)}
rfc_grid = GridSearchCV(rfc, parameters, cv=3, scoring='f1')
rfc_grid.fit(train_tfidf, target_train)
rfc_grid.best_params_

CPU times: user 4min 6s, sys: 0 ns, total: 4min 6s
Wall time: 4min 8s


{'max_depth': 9,
 'min_samples_leaf': 3,
 'min_samples_split': 2,
 'n_estimators': 45}

 Вычислим значение f1 на тестовой выборке при max_depth = 9, min_samples_leaf = 3, min_samples_split = 2 и n_estimators = 45.

In [68]:
%%time

rfc = RandomForestClassifier(max_depth=9, min_samples_leaf=3, min_samples_split=2, n_estimators=45, class_weight='balanced', random_state=12345) 
rfc.fit(train_tfidf, target_train)
predictions = rfc.predict(test_tfidf)
print('Значение f1 при использовании модели случайного леса:', round(f1_score(target_test, predictions), 2))

Значение f1 при использовании модели случайного леса: 0.38
CPU times: user 476 ms, sys: 0 ns, total: 476 ms
Wall time: 486 ms


### Модель градиентного бустинга

Найдем оптимальные гиперпараметры для модели градиентного бустинга.

In [69]:
%%time

xgbc = XGBClassifier(random_state=12345)
parametrs = {'max_depth': range(1, 10, 2),
            'learning_rate':[0.1, 0.3],
            'n_estimators': range(10, 50, 10)}
xgbc_grid = GridSearchCV(xgbc, parametrs, scoring='f1', cv=3)
xgbc_grid.fit(train_tfidf, target_train)
xgbc_grid.best_params_

/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:14:30] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:14:33] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:52:34] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:52:58] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
CPU times: user 38min 37s, sys: 8.35 s, total: 38min 45s
Wall time: 39min 1s


{'learning_rate': 0.3, 'max_depth': 9, 'n_estimators': 40}

Вычислим значение f1 на тестовой выборке при learning_rate =  0.3, max_depth = 9 и n_estimators = 40.

In [70]:
%%time

xgbc = XGBClassifier(learning_rate=0.3, max_depth=9, n_estimators = 40, random_state=12345) 
xgbc.fit(train_tfidf, target_train)
predictions = xgbc.predict(test_tfidf)
print('Значение f1 при использовании модели градиентного бустинга:', round(f1_score(target_test, predictions), 2))

[16:53:33] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Значение f1 при использовании модели градиентного бустинга: 0.67
CPU times: user 55.1 s, sys: 187 ms, total: 55.2 s
Wall time: 55.8 s


## Выводы

Достижение поставленной руководством компании "Викишоп" цели осуществлено в два этапа:

1. Предобработка текста.
2. Обучение моделей.

На первом этапе текст удалось выполнить следующие задачи:

- привести все буквы к нижнему регистру;
- очистить текст от ненужных символов;
- провести токенизацию и лемматизацию текста.

Для эффективного обучения моделей была осуществлена оценка важности слов величиной TF-IDF, которую мы использовали для трансформации фичей. В качестве обучаемых моделей были выбраны:

1. Модель логистической регрессии.
2. Модель решающего дерева.
3. Модель случайного леса.
4. Модель градиентного бустинга.

В качестве метрики руководство компании установило значение F1 не ниже 0.75. Лишь модель логистической регрессии смогла достигнуть планового показателя. Помимо качества модели стоит отметить ее скорость: она уступила только модели решающего дерева. 

Таким образом, составленная модель логистической регрессии позволит компании качественно классифицировать комментарии на позитивыне и негативные.